Installers<br>
pandas: !pip install pandas<br>
K-means: !pip install -U scikit-learn<br>
Matplotlib: !python -m pip install -U matplotlib<br>
nltk: !pip install nltk<br>
PySimpleGUI: !pip install pysimplegui

In [21]:
from asyncore import write
import pandas as pd
from doctest import DocTestFailure
import math
import string
from sklearn.cluster import KMeans
from sklearn.datasets._samples_generator import make_blobs

#----------- importando bibliotecas para tratar as palavras

#from worldcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from nltk.tokenize import word_tokenize
import nltk

#----------- Biblioteca para interface
import PySimpleGUI as sg

In [22]:
#----------- funções

def remove_punctuations(line):
    for character in string.punctuation:
        line = line.replace(character, "")
    return line

def writeFile(dict, archiveName):
    arquivo = open(archiveName+".txt",'w')
    for word, tf in dict.items():
        sentence = f"{word}:{tf}\n"
        arquivo.write(sentence)
    arquivo.close()

def writeCSV(df, archiveName):
    f = open("..\\DataBase\\"+archiveName+".csv", 'w', newline='', encoding="utf8")
    df.to_csv("..\\DataBase\\"+archiveName+".csv", encoding="utf8", sep=';')


def countWords(wordsDoc, line): 
    wordsCopy = wordsDoc.copy() 
    words = line.split()
    for word in words:
        word = word.lower()
        wordsCopy[word] += 1
    return wordsCopy

def get_posts():
    file = open(filepath, 'r', encoding="utf8")
    lines = file.readlines()
    return lines  

In [23]:
#------------- funções de ponderação/vetorização

def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = round(count/float(corpusCount), 3)
    return(tfDict)

def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    #idfDict = dict.fromkeys(docList[1].keys, 0)
    for word, val in docList.items():
        idfDict[word] = round(math.log10(N / (float(val) + 1)), 3)
        
    return(idfDict)

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = round(val*idfs[word], 3)
    return(tfidf)

In [24]:
#--------- pegando cada palavra dos posts 

filepath = "..\\DataBase\\Governamentais.txt"
word_count = {}
lines = []


with open(filepath, 'r', encoding="utf8") as fi:
    for line in fi:
        line = remove_punctuations(line)
        lines.append(line)
        words = line.split()

        for word in words:
            word = word.lower()
            if word not in word_count:
                word_count[word] = 0

In [25]:
#-----criando listas auxiliares
listTF = {}
listIDF = {}
listTF_IDF = {}
tweets = {}

In [26]:
#-----ponderação/vetorização
rowCount = 0
for line in lines:
    rowCount += 1
    rowName = str(rowCount)
    totalCount = countWords(word_count, line)
    totalTF = computeTF(totalCount, line)
    listTF[rowName] = totalTF
    totalIDF = computeIDF(totalCount)
    listIDF[rowName] = totalIDF
    totalTF_IDF = computeTFIDF(totalCount, totalIDF)
    listTF_IDF[rowName] = totalTF_IDF
    tweets[rowName] = line

In [27]:
#------------ criando os dataframes

dfTF = pd.DataFrame.from_dict(listTF, orient='index', columns=word_count.keys())
dfIDF = pd.DataFrame.from_dict(listIDF, orient='index', columns=word_count.keys())
dfTF_IDF = pd.DataFrame.from_dict(listTF_IDF, orient='index', columns=word_count.keys())
dfTweets = pd.DataFrame.from_dict(tweets, orient='index', columns=['TEXT'])


In [8]:
#---------escrevendo no CSV
# Só execute caso queira gerar arquivos novos com novos posts
#writeCSV(dfTweets, "Relação Tweets com Dataframe")
#writeCSV(dfTF, "TF")
#writeCSV(dfIDF, "IDF")
#writeCSV(dfTF_IDF, "TF_IDF")

In [28]:
#---------clusterização --- TF
def clusterizar(dfTF):
    x = np.array(dfTF)
    kmeans_model = KMeans (n_clusters=3, random_state=0, init='random', n_init=10, max_iter=300, tol=1e-04).fit(x)
    kmeans_model.labels_
    kmeans_model.cluster_centers_

    mydict = {i: np.where(kmeans_model.labels_ == i)[0] for i in range(kmeans_model.n_clusters)}
    dictlist = []
    for key, value in mydict.items():
        temp = [key,value]
        dictlist.append(temp)
        
    return dictlist

In [29]:
dictlist = clusterizar(dfTF)
display(dictlist)

[[0,
  array([  7,   9,  11,  14,  19,  23,  32,  48,  50,  60,  61,  66,  68,
          72,  73,  75,  77,  78,  79,  97,  98,  99, 108, 110, 112, 114,
         125, 130, 139, 142, 149, 151, 152, 156, 157, 159, 164, 169, 170,
         173, 182, 184, 186, 188, 194, 197, 199, 202, 203, 205, 208, 215,
         217, 221, 231, 236, 237, 239, 240, 242, 245, 257, 264, 268, 272,
         277, 282, 283, 290, 306, 344, 352, 357, 362, 372, 379, 397, 403,
         405, 426, 434, 436, 444, 449, 451, 456, 458, 459, 461, 465, 466,
         475, 477, 479, 480, 485, 492, 495, 496, 504, 508, 523, 534, 539,
         542, 543, 548, 550, 557, 559, 567, 568, 569, 570, 573, 574, 575,
         577, 580, 593, 607, 613, 625, 668, 671, 682, 704, 705, 720, 748,
         754, 755, 756, 760, 761, 762, 765, 771, 778, 785, 792, 799, 806,
         813, 820, 827, 834, 841], dtype=int64)],
 [1,
  array([549, 630, 650, 653, 659, 661, 665, 674, 676, 678, 680, 681, 683,
         684, 685, 687, 688, 706, 710, 711, 716, 717

### Quantidade de posts no grupo 0

In [11]:
len(dictlist[0][1])

148

### Quantidade de posts no grupo 1

In [12]:
len(dictlist[1][1])

30

### Quantidade de posts no grupo 2

In [13]:
len(dictlist[2][1])

668

### Inserir coluna do grupo

In [30]:
#--------- Função para inserir coluna dos grupos
def insert_column(df_aux):
    for i in dictlist[0][1]:
        df_aux['group'][i-1] = 0

    for i in dictlist[1][1]:
        df_aux['group'][i-1] = 1

    for i in dictlist[2][1]:
        df_aux['group'][i-1] = 2
    
    return df_aux

In [31]:
df_aux = dfTF
df_aux['group'] = ''
   
df_aux = insert_column(df_aux)
display(df_aux)

C:\Users\Jair\AppData\Local\Temp\ipykernel_8708\1268836670.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['group'][i-1] = 0
C:\Users\Jair\AppData\Local\Temp\ipykernel_8708\1268836670.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['group'][i-1] = 1
C:\Users\Jair\AppData\Local\Temp\ipykernel_8708\1268836670.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['group'][i-1] = 2


,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,óculos,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2


## Classificar as amostras

### Função para classificação

In [33]:
# A função retorna verdadeiro caso todas as amostras sejam classificadas igualmente
def classification(amostra,lines):    
    sg.theme('DarkAmber')

    first = True
    equals = True

    for i in amostra.index:
        layout = [  [sg.Text(lines[int(i)-1])],
                    [sg.Button('Positivo'), sg.Button('Negativo'), sg.Button('Neutro')] ]

        window = sg.Window('Classificação de Post', layout)

        event, values = window.read()
        if(event == sg.WIN_CLOSED):
            break
        elif(first):
            classification = event
            first = False
        elif(classification != event):
            equals = False
            break

        window.close()
    window.close()    
    return equals


### Pegar posts como amostra de cada grupo

In [34]:
def get_group(grupo):
    return df_aux[df_aux['group'] == grupo]

### Pega os posts em txt

In [35]:
lines = get_posts()

### Funções para classificar os grupos

In [40]:
def classify_gp0():
    gp0 = get_group(0)
    result = classification(gp0.sample(5), lines)
    if(not result):
        print('Há posts com classifcação diferente. Será necessário efetuar a clusterização para os posts do grupo 0!')
        dictlist = clusterizar(gp0)
        df_aux = insert_column(df_aux)
        classify_gp0()
    else:
        print('O grupo 0 está OK!')    

In [ ]:
def classify_gp1():
    gp1 = get_group(1)
    result = classification(gp1.sample(5), lines)
    if(!result):
        print('Há posts com classifcação diferente. Será necessário efetuar a clusterização para os posts do grupo 1!')
        dictlist = clusterizar(gp1)
        df_aux = insert_column(df_aux)
        classify_gp1()
    else:
        print('O grupo 1 está OK!')    

In [ ]:
def classify_gp2():
    gp2 = get_group(2)
    result = classification(gp2.sample(5), lines)
    if(!result):
        print('Há posts com classifcação diferente. Será necessário efetuar a clusterização para os posts do grupo 2!')
        dictlist = clusterizar(gp2)
        df_aux = insert_column(df_aux)
        classify_gp2()
    else:
        print('O grupo 2 está OK!')    

### Classificar Grupo 0

In [41]:
classify_gp0()

Há posts com classifcação diferente. Será necessário efetuar a clusterização para os posts do grupo 0!


UnboundLocalError: local variable 'df_aux' referenced before assignment

### Classificar Grupo 1

In [ ]:
classify_gp1()

### Classificar Grupo 2

In [ ]:
classify_gp2()